In [1]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

C:\Users\Andre\anaconda3\envs\EngML\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [19]:
eids, dfs = utils.get_twitter_from_dir("data/Twitter/", columns=["author_id", "created_at", "text"])

In [68]:
eids_, labels, _ = utils.get_twitter_conversations("data/Twitter.txt")
eid_label = {}
for eid, label in zip(eids_, labels):
    eid_label[eid] = label
with open("models/eid_label_dict.json", "w") as file:
    json.dump(eid_label, file, indent=4)

In [3]:
model = Doc2Vec.load("models/doc2vec_gensim_100.model")
with open("models/user_dict.json", "r") as file:
    user_dict = json.load(file)
with open("models/eid_label_dict.json", "r") as file:
    eid_label_dict = json.load(file)
with np.load("models/user_matrices.npz") as mats:
    user_mat = mats["user_mat"]
    user_bin_mat = mats["user_bin_mat"]

In [20]:
def get_document(texts, tokens_only=False):
    for i, text in enumerate(texts):
        tokens = gensim.utils.simple_preprocess(text)
        if tokens_only:
            yield tokens
        else:
            yield TaggedDocument(tokens, [i])

def add_embedding(df, doc2word_model):
    tokens = list(get_document(df["text"].to_list(), True))
    embeddings = [doc2word_model.infer_vector(tweet_tokens) for tweet_tokens in tokens]
    df["embedding"] = embeddings
    return df

def add_author(df, user_dict):
    df["author"] = df["author_id"].apply(lambda x: user_dict[str(x)])
    return df

def add_user_information(df, user_bin_mat):
    df["author_info"] = df["author"].apply(lambda x: user_bin_mat[x])
    return df

def to_model_input(df):
    foo = df[["bin", "timedelta_previous_bin", "embedding", "author_info"]]\
        .groupby(by="bin")\
        .agg(count=("bin", "size"),
             timedelta_previous_bin=("timedelta_previous_bin", "max"),
             embedding=("embedding", "mean"),
             author_info=("author_info", "mean"))\
        .reset_index()
    return np.vstack(foo.drop("bin", axis=1).apply(lambda x: np.hstack(x.values), axis=1).values)

def to_model_user_input(df, user_mat):
    return np.vstack(df["author"].apply(lambda x: user_mat[x]).values)


model_data = []
bin_size = 60 * 60 * 12 #  12 hours
threshold = 2 * 90 #  180 bins = 180 * 12 hours = 90 days
for i, (eid, df) in enumerate(zip(eids, dfs)):
    utils.cut_bins(utils.add_bins(utils.timedelta(df), bin_size), threshold)
    add_embedding(df, model)
    add_author(df, user_dict)
    add_user_information(df, user_bin_mat)
    model_data.append([eid, eid_label_dict[eid], to_model_input(df), to_model_user_input(df, user_mat)])

In [21]:
len(model_data)

992

In [22]:
dataset_df = pd.DataFrame(model_data, columns=["eid", "label", "x", "y"])

In [29]:
data_np = dataset_df.to_numpy()

In [40]:
data_np

array([['Airfrance', '0',
        array([[ 3.10000000e+01,  0.00000000e+00, -4.93319966e-02,
                 1.00782543e-01,  2.95423009e-02, -1.01779560e-02,
                 3.57322395e-02,  3.59436322e-04,  5.23099564e-02,
                 1.17184278e-02, -1.00474775e-01, -1.73111334e-01,
                 3.16258450e-03,  1.29577085e-01, -1.89070422e-02,
                -1.46031439e-01,  6.31916374e-02, -6.62704855e-02,
                 1.37676522e-01,  3.69293578e-02,  2.02653874e-02,
                -5.75482138e-02, -3.46826725e-02, -2.68886723e-02,
                 4.73972969e-03,  7.42756426e-02, -4.37553711e-02,
                -4.81052361e-02,  3.84620354e-02,  1.63630813e-01,
                 1.33463085e-01, -8.97835381e-03, -2.35538296e-02,
                -2.02563498e-02, -1.32352626e-02,  9.40895826e-02,
                 1.16482131e-01,  2.61853840e-02, -3.51646938e-03,
                 3.44978310e-02, -1.30386967e-02, -3.66349667e-02,
                 5.88158630e-02,  7.

In [27]:
dataset_df.to_csv("data/processed/data_bin12h_cut180_100w_20xu_50yu.npz", index=False)

In [36]:
np.save("data/processed/data_bin12h_cut180_100w_20xu_50yu.npy", dataset_df.to_numpy())

In [39]:
pd.DataFrame(np.load("data/processed/data_bin12h_cut180_100w_20xu_50yu.npy", allow_pickle=True))

,0,1,2,3
0,Airfrance,0,"[[31.0, 0.0, -0.04933199658989906, 0.100782543...","[[5.728417000232961e-07, 7.497009531531812e-12..."
1,Airliner,0,"[[35.0, 0.0, -0.09998174756765366, 0.010489981...","[[2.8766544422301893e-07, 6.536015973668471e-1..."
2,Amanda,0,"[[2.0, 0.0, -0.022407064214348793, -0.11954444...","[[0.0002461661780663931, 3.0744367038198896e-0..."
3,AnnieLe,0,"[[1.0, 0.0, 0.06573211401700974, 0.16498757898...","[[1.6659823040681458e-10, 2.4046260842342657e-..."
4,BarnesNobleObamaMonkey,0,"[[2.0, 0.0, -0.08572602272033691, 0.1293545961...","[[3.3745972849026395e-06, 2.2055798996612257e-..."
...,...,...,...,...
987,Turkishcrash,0,"[[596.0, 0.0, -0.018301477655768394, 0.0502989...","[[3.328299035345699e-06, 8.289281424581946e-11..."
988,twittersummize,0,"[[45.0, 0.0, -0.003588458988815546, 0.11465315...","[[4.780339256276481e-07, 8.799779182761624e-10..."
989,Vanessa,0,"[[22.0, 0.0, 0.03274635970592499, 0.1191900223...","[[0.003943954130925907, 1.7515275933934912e-06..."
990,WesternSpaghetti,0,"[[1.0, 0.0, -0.0018492384115234017, 0.08881287...","[[1.3914953201835611e-10, 1.5261085385096275e-..."
